In [40]:
def ms3_FeatureCompute(input_path, output_path):

    import os

    import numpy
    import pandas as pd
    import math
    
    # specify path and file names
    #user_input1 = input("Enter the name of target folder: ")

    #user_input2 = input("Enter the name of target CSV: ")
    #pth = "Z:\\live\\sub-42-o\\cameras\\kinematic\\calib\\"
    #path = pth + user_input2 + ".csv"
    
    for file_name in os.listdir(input_path):
        if file_name.endswith('.csv'):
            full_file_path = os.path.join(input_path, file_name)
            print("Processing file:", file_name)

            prep = pd.read_csv(full_file_path, index_col=0)
            ### feature computations#
            def Angle(v1, v2):  # angle w/o direction

                    cos_theta_v1_v2 = numpy.dot(v1, v2) / (numpy.linalg.norm(v1) * numpy.linalg.norm(v2))  # get cosine
                    theta_v1_v2 = numpy.degrees(numpy.arccos(cos_theta_v1_v2))

                    return theta_v1_v2

                ##
            def ShortestDistance(line_point1, line_point2, point):
                AB = numpy.subtract(line_point2, line_point1)
                AC = numpy.subtract(point, line_point1)

                area = numpy.linalg.norm(numpy.cross(list(AB), list(AC)))
                CD = area / numpy.linalg.norm(AB)

                return CD

            ##
            def CurveArea(neck, pelvis, chest, navel):
                distance_chest = ShortestDistance(neck, pelvis, chest)
                distance_navel = ShortestDistance(neck, pelvis, navel)

                #
                base_chest = math.sqrt((numpy.linalg.norm(numpy.subtract(neck, chest))) ** 2 - distance_chest ** 2)
                base_navel = math.sqrt((numpy.linalg.norm(numpy.subtract(pelvis, navel))) ** 2 - distance_navel ** 2)

                #
                upper_body = numpy.linalg.norm(numpy.subtract(neck, pelvis))

                #
                Curve_Area = (base_chest * distance_chest / 2 + base_navel * distance_navel / 2 + (
                        upper_body - base_chest - base_navel) * (distance_chest + distance_navel) / 2) / upper_body

                return Curve_Area

            def AnkDis(left_ank, right_ank):
                ankle_dis = numpy.linalg.norm(right_ank - left_ank)

                return ankle_dis

            def GetNewAxesDirection(right_cla, left_cla, right_hip, left_hip, nose):
                cla_vec = right_cla - left_cla
                pelvis_vec = right_hip - left_hip  # new x_axis; left to right

                xn = pelvis_vec

                # midpoint of shoulder
                cla_mid = (right_cla + left_cla) / 2

                # get normal vec (new z_axis (front-back)) of the plane containing
                # pelvis and cla midpoint (new xy-plane)
                vec_mid = cla_mid - left_hip
                zn = numpy.cross(pelvis_vec, vec_mid)  # new z_axis; direction pending

                # cross product b/w new x- and z-axes to get y-axis
                yn = numpy.cross(zn, xn)  # new y_axis


                if numpy.dot(yn, right_cla - left_hip) < 0:
                    print("*** warning: yn direction flipped ***")
                    yn = -yn

                # determine direction of new z_axis
                z_ref = numpy.dot(zn, nose - (left_hip + right_hip) / 2)

                # assess the cosine b/w z(front back) and pelvis-nose vector
                # if dot product < 0, flip z direction to ensure nose is at front
                if z_ref < 0:
                    zn = -zn

                return xn, yn, zn

            ##
            def ShoulderSlope_drct(right_shoulder, left_shoulder, right_hip, left_hip, xn, yn, zn):
                shoulder_vec = right_shoulder - left_shoulder
                pelvis_vec = right_hip - left_hip  # new x_axis; left to right

                # compute up-down orientation
                proj_xy = shoulder_vec - numpy.dot(shoulder_vec, zn) * zn / numpy.linalg.norm(
                    zn ** 2)  # projection of shoulder vec on x-y plane
                cos_theta_xy = numpy.dot(xn, proj_xy) / (numpy.linalg.norm(xn) * numpy.linalg.norm(proj_xy))
                theta_xy = numpy.degrees(numpy.arccos(cos_theta_xy))

                if numpy.dot(yn, shoulder_vec) > 0:
                    right_up = 1
                elif numpy.dot(yn, shoulder_vec) < 0:
                    right_up = -1
                elif numpy.dot(yn, shoulder_vec) == 0:
                    right_up = 0
                else:
                    right_up = numpy.nan

                    # compute front-back orientation
                proj_xz = shoulder_vec - numpy.dot(shoulder_vec, yn) * yn / numpy.linalg.norm(yn ** 2)
                cos_theta_xz = numpy.dot(xn, proj_xz) / (numpy.linalg.norm(xn) * numpy.linalg.norm(proj_xz))
                theta_xz = numpy.degrees(numpy.arccos(cos_theta_xz))

                if numpy.dot(zn, shoulder_vec) > 0:
                    right_front = 1
                elif numpy.dot(zn, shoulder_vec) < 0:
                    right_front = -1
                elif numpy.dot(zn, shoulder_vec) == 0:
                    right_front = 0
                else:
                    right_front = numpy.nan

                return theta_xy, right_up, theta_xz, right_front

            def AnkleDistance_drct(right_ank, left_ank, zn):
                ank_vec = right_ank - left_ank

                # compute front-back orientation
                if numpy.dot(zn, ank_vec) > 0:
                    rightAnk_front = 1
                elif numpy.dot(zn, ank_vec) < 0:
                    rightAnk_front = -1
                elif numpy.dot(zn, ank_vec) == 0:
                    rightAnk_front = 0
                else:
                    rightAnk_front = numpy.nan

                return rightAnk_front

            # HELPER FUNCTIONS CENTER OF MASS

            def calculate_midpoint(point1, point2):
                midpoint = (point1 + point2) / 2
                return midpoint

            def point_distance(point1, point2):
                distance = numpy.linalg.norm(point1 - point2)

                return distance

            def get_line(point1, point2):
                x1, y1, z1 = point1
                x2, y2, z2 = point2

                # Calculate the differences between the coordinates
                dx = x2 - x1
                dy = y2 - y1
                dz = z2 - z1

                # Calculate the coefficients A, B, and C
                A = dy
                B = -dx
                C = dx * z1 - dz * x1

                # Normalize the coefficients to have the same sign (optional step)
                if A < 0:
                    A = -A
                    B = -B
                    C = -C

                # Calculate the constant D
                D = -(A * x1 + B * y1 + C * z1)
                line = np.array([A, B, C, D])

                return line

            def calculate_plane_coefficients(x1, y1, z1, x2, y2, z2, x3, y3, z3):
                # Calculate the vectors
                vector1 = (x2 - x1, y2 - y1, z2 - z1)
                vector2 = (x3 - x1, y3 - y1, z3 - z1)

                # Calculate the normal vector
                A = vector1[1] * vector2[2] - vector1[2] * vector2[1]
                B = vector1[2] * vector2[0] - vector1[0] * vector2[2]
                C = vector1[0] * vector2[1] - vector1[1] * vector2[0]

                # Find the D coefficient
                D = -A * x1 - B * y1 - C * z1
                plane = np.array([A, B, C, D])
                return plane

            def project_point_to_plane(point, plane):
                # Normalize the plane normal vector
                plane_normal = plane[:3] / np.linalg.norm(plane[:3])

                # Calculate the distance from the point to the plane
                distance_to_plane = np.dot(point, plane_normal) + plane[3]

                # Calculate the projected point
                projected_point = point - distance_to_plane * plane_normal

                return projected_point

            def distance_to_line(point, line_coefficients):
                x0, y0, z0 = point
                A, B, C, D = line_coefficients

                # Calculate the distance using the formula
                distance = abs(A * x0 + B * y0 + C * z0 + D) / np.sqrt(A ** 2 + B ** 2 + C ** 2)

                return distance

            def COM_FrontBack(plane, foot_left, ankle_left, foot_right, ankle_right, projected_point):

                # first, we project all points to the same plane

                foot_left = project_point_to_plane(foot_left, plane)
                ankle_left = project_point_to_plane(ankle_left, plane)
                foot_right = project_point_to_plane(foot_right, plane)
                ankle_right = project_point_to_plane(ankle_right, plane)

                # Obtain midpoints within each foot, between tip of foot (Foot_left) and ankles
                mid_left = calculate_midpoint(foot_left, ankle_left)
                mid_right = calculate_midpoint(foot_right, ankle_right)

                # Obtain midpoints between the top of the feet, and separetely bewteen the ankles
                mid_feet = calculate_midpoint(foot_left, foot_right)
                mid_ankles = calculate_midpoint(ankle_left, ankle_right)

                # Obtain midpoints between the top of the feet, and separetely bewteen the ankles
                mid_feet = calculate_midpoint(foot_left, foot_right)
                mid_ankles = calculate_midpoint(ankle_left, ankle_right)

                # Lenght of feet to normalize after
                len_vertical = (numpy.linalg.norm(mid_feet - mid_ankles))
                len_horizontal = (numpy.linalg.norm(mid_left - mid_right))

                # Get longest length, either right foot to left ankle or left foot to right ankle, whichever one is bigger, that will be the normalization for front back
                len_RF_LA = (numpy.linalg.norm(foot_right - ankle_left))
                len_LF_RA = (numpy.linalg.norm(foot_left - ankle_right))
                if len_RF_LA > len_LF_RA:
                    normalizer = len_RF_LA
                else:
                    normalizer = len_LF_RA

                # Line equation between mid points
                vertical = get_line(mid_feet, mid_ankles)
                horizontal = get_line(mid_left, mid_right)

                # Is it front or back?
                # If the distance from the projected point to the midfeet is smaller than the distance between the projected point and the midankles point, then it is more forwards than backwards

                if (point_distance(projected_point, mid_ankles)) < (point_distance(projected_point, mid_ankles)):
                    a = 1
                else:
                    a = 0

                # Distance between projected point and horizontal (will tell us how front or back the projected point is in respect from the center between the feet)
                # This value has to be normalized by the lengh of the vertical or horizontal line
                dis = distance_to_line(projected_point, horizontal)
                dis_norm = dis / normalizer

                # If point is more front, then a = 1, so we put a positive value, if it is 0, then negative value

                if a == 1:
                    dis_norm = dis_norm
                elif a == 0:
                    dis_norm = -dis_norm

                return dis_norm

            def COM_LeftRight(plane, foot_left, ankle_left, foot_right, ankle_right, projected_point):

                # first, we project all points to the same plane

                foot_left = project_point_to_plane(foot_left, plane)
                ankle_left = project_point_to_plane(ankle_left, plane)
                foot_right = project_point_to_plane(foot_right, plane)
                ankle_right = project_point_to_plane(ankle_right, plane)

                # Obtain midpoints within each foot, between tip of foot (Foot_left) and ankles
                mid_left = calculate_midpoint(foot_left, ankle_left)
                mid_right = calculate_midpoint(foot_right, ankle_right)

                # Obtain midpoints between the top of the feet, and separetely bewteen the ankles
                mid_feet = calculate_midpoint(foot_left, foot_right)
                mid_ankles = calculate_midpoint(ankle_left, ankle_right)

                # Lengh of feet to normalize after
                len_vertical = (numpy.linalg.norm(mid_feet - mid_ankles))
                len_horizontal = (numpy.linalg.norm(mid_left - mid_right))

                # Line equation between mid points
                vertical = get_line(mid_feet, mid_ankles)
                horizontal = get_line(mid_left, mid_right)

                # Normalization will be done with the longest between either foot left foot right distance
                foot_foot = (numpy.linalg.norm(foot_left - foot_right))
                ankle_ankle = (numpy.linalg.norm(ankle_left - ankle_right))

                if foot_foot > ankle_ankle:
                    normalizator = foot_foot
                else:
                    normalizator = ankle_ankle


                # Is it left or right in respect to the center point?
                # If the distance from the projected point to the midfeet is smaller than the distance between the projected point and the midankles point, then it is more forwards than backwards

                if (point_distance(projected_point, mid_left)) < (point_distance(projected_point, mid_right)):
                    a = 1
                else:
                    a = 0

                # Distance between projected point and vertical (will tell us how front or back the projected point is in respect from the center between the feet)
                # This value has to be normalized by the lengh of the vertical or horizontal line
                dis = distance_to_line(projected_point, vertical)
                dis_norm = dis / normalizator

                # If point is more front, then a = 1, so we put a positive value, if it is 0, then negative value

                if a == 1:
                    dis_norm = dis_norm
                elif a == 0:
                    dis_norm = -dis_norm

                return dis_norm

            # Hacemoos la media de las x, de las y y de las z por separado de cada punto
            import numpy as np
            # Media de las x:
            x_foot_right = prep["FOOT_RIGHT_x"].mean()
            y_foot_right = prep["FOOT_RIGHT_y"].mean()
            z_foot_right = prep["FOOT_RIGHT_z"].mean()

            # FOOT LEFT
            x_foot_left = prep["FOOT_LEFT_x"].mean()
            y_foot_left = prep["FOOT_LEFT_y"].mean()
            z_foot_left = prep["FOOT_LEFT_z"].mean()

            # ANKLE LEFT
            x_ankle_left = prep["ANKLE_LEFT_x"].mean()
            y_ankle_left = prep["ANKLE_LEFT_y"].mean()
            z_ankle_left = prep["ANKLE_LEFT_z"].mean()

            # Obtenemos el punto medio de cada uno y con eso determinamos el plano del suelo
            plane = calculate_plane_coefficients(x_foot_right, y_foot_right, z_foot_right, x_foot_left, y_foot_left,
                                                 z_foot_left, x_ankle_left, y_ankle_left, z_ankle_left)

            # print(f"The equation of the plane is: {A}x + {B}y + {C}z + {D} = 0")

            ##
            def Norm_WristHip_Left(left_hip, left_shoulder, left_elbow, left_wrist):
                lenght_left = (numpy.linalg.norm(left_shoulder - left_elbow)) + (numpy.linalg.norm(left_elbow - left_wrist))

                return lenght_left

            ##
            def Norm_WristHip_Right(right_hip, right_shoulder, right_elbow, right_wrist):
                lenght_right = (numpy.linalg.norm(right_shoulder - right_elbow)) + (numpy.linalg.norm(right_elbow - right_wrist))

                return lenght_right

            ##
            def WristHipLeft(left_hip, left_shoulder, left_elbow, left_wrist):
                dis = numpy.linalg.norm(left_hip - left_wrist)
                norm = Norm_WristHip_Left(left_hip, left_shoulder, left_elbow, left_wrist)
                dis = dis / norm

                return dis

            ##
            def WristHipRight(right_hip, right_shoulder, right_elbow, right_wrist):
                dis = numpy.linalg.norm(right_wrist - right_hip)
                norm = Norm_WristHip_Right(right_hip, right_shoulder, right_elbow, right_wrist)
                dis = dis / norm

                return dis

            ## compute time series

            def compute_features(row):
                right_shoulder = numpy.array(row['SHOULDER_RIGHT_x':'SHOULDER_RIGHT_z'])
                left_shoulder = numpy.array(row['SHOULDER_LEFT_x':'SHOULDER_LEFT_z'])

                right_hip = numpy.array(row['HIP_RIGHT_x':'HIP_RIGHT_z'])
                left_hip = numpy.array(row['HIP_LEFT_x':'HIP_LEFT_z'])

                neck = numpy.array(row['NECK_x':'NECK_z'])
                right_cla = numpy.array(row['CLAVICLE_RIGHT_x':'CLAVICLE_RIGHT_z'])
                left_cla = numpy.array(row['CLAVICLE_LEFT_x':'CLAVICLE_LEFT_z'])

                nose = numpy.array(row['NOSE_x':'NOSE_z'])
                pelvis = numpy.array(row['PELVIS_x':'PELVIS_z'])

                spine_chest = numpy.array(row['SPINE_CHEST_x':'SPINE_CHEST_z'])
                spine_naval = numpy.array(row['SPINE_NAVEL_x':'SPINE_NAVEL_z'])

                left_knee = numpy.array(row['KNEE_LEFT_x':'KNEE_LEFT_z'])
                right_knee = numpy.array(row['KNEE_RIGHT_x':'KNEE_RIGHT_z'])

                left_ank = numpy.array(row['ANKLE_LEFT_x':'ANKLE_LEFT_z'])
                right_ank = numpy.array(row['ANKLE_RIGHT_x':'ANKLE_RIGHT_z'])

                right_elbow = numpy.array(row['ELBOW_RIGHT_x':'ELBOW_RIGHT_z'])
                left_elbow = numpy.array(row['ELBOW_LEFT_x':'ELBOW_LEFT_z'])

                right_shoulder = numpy.array(row['SHOULDER_RIGHT_x':'SHOULDER_RIGHT_z'])
                left_shoulder = numpy.array(row['SHOULDER_LEFT_x':'SHOULDER_LEFT_z'])

                left_foot = numpy.array(row['FOOT_LEFT_x':'FOOT_LEFT_z'])
                right_foot = numpy.array(row['FOOT_RIGHT_x':'FOOT_RIGHT_z'])

                left_wrist = numpy.array(row['WRIST_LEFT_x':'WRIST_LEFT_z'])
                right_wrist = numpy.array(row['WRIST_RIGHT_x':'WRIST_RIGHT_z'])

                left_ank = numpy.array(row['ANKLE_LEFT_x':'ANKLE_LEFT_z'])
                right_ank = numpy.array(row['ANKLE_RIGHT_x':'ANKLE_RIGHT_z'])
                ### w/o direction
                ## trunk angle
                upper_vec = neck - pelvis
                lower_vec = (left_knee + right_knee) / 2 - pelvis

                trunk_angle = Angle(upper_vec, lower_vec)

                ## shoulder slope w/o direction
                shoulder_vec = right_shoulder - left_shoulder
                pelvis_vec = right_hip - left_hip

                shoulder_slope_no_direction = Angle(shoulder_vec, pelvis_vec)

                ## spine curve
                spine_curvature = CurveArea(neck, pelvis, spine_chest, spine_naval)

                ## ankle distance
                ankle_distance = AnkDis(left_ank, right_ank)

                ### with direction
                xn, yn, zn = GetNewAxesDirection(right_cla, left_cla, right_hip, left_hip, nose)

                ## shoulder slope with directions
                shoulder_slope_up_down, shoulder_slope_right_up, shoulder_slope_front_back, shoulder_slope_right_front = ShoulderSlope_drct(
                    right_shoulder, left_shoulder,
                    right_hip, left_hip, xn, yn, zn)

                ## ankle distance direction
                ankle_distance_right_front = AnkleDistance_drct(right_ank, left_ank, zn)

                ## Distance from right hip to right wrist normalized
                hip_wrist_dist_right = WristHipRight(right_hip, right_shoulder, right_elbow, right_wrist)

                ## Distance from left hip to left wrist normalized
                hip_wrist_dist_left = WristHipLeft(left_hip, left_shoulder, left_elbow, left_wrist)

                point = calculate_midpoint(right_hip, left_hip)
                projected_point = project_point_to_plane(point, plane)
                COM_left_right = COM_LeftRight(plane, left_foot, left_ank, right_foot, right_ank, projected_point)
                COM_front_back = COM_FrontBack(plane, left_foot, left_ank, right_foot, right_ank, projected_point)

                ## ankle distance direction ref
                #     left_foot = numpy.array(row['FOOT_LEFT_x':'FOOT_LEFT_z'])
                #     right_foot = numpy.array(row['FOOT_RIGHT_x':'FOOT_RIGHT_z'])

                #     dis_right = numpy.linalg.norm(right_foot - left_ank)
                #     dis_left = numpy.linalg.norm(left_foot - right_ank)

                #     if dis_right > dis_left:
                #         ankle_distance_right_front2 = 1
                #     elif dis_right < dis_left:
                #         ankle_distance_right_front2 = -1
                #     elif dis_right == dis_left:
                #         ankle_distance_right_front2 = 0
                #     else:
                #         ankle_distance_right_front2 = numpy.nan

                #####
                features = numpy.array([shoulder_slope_no_direction,
                                        spine_curvature,
                                        trunk_angle,
                                        ankle_distance,
                                        shoulder_slope_up_down, shoulder_slope_right_up,
                                        shoulder_slope_front_back, shoulder_slope_right_front,
                                        ankle_distance_right_front, hip_wrist_dist_right, hip_wrist_dist_left,
                                        COM_front_back, COM_left_right])

                return features

            # compute
            feat = numpy.empty(shape=[len(prep), 13])

            import time as T

            start_time = T.time()

            for r in range(len(prep)):

                if r == int(len(prep) / 10):
                    print('10% completed')
                    print("--- %s seconds ---" % (T.time() - start_time))

                if r == int(len(prep) / 4):
                    print('25% completed')
                    print("--- %s seconds ---" % (T.time() - start_time))

                if r == int(len(prep) / 2):
                    print('50% completed')
                    print("--- %s seconds ---" % (T.time() - start_time))

                if r == int((len(prep) / 4) * 3):
                    print('75% completed')
                    print("--- %s seconds ---" % (T.time() - start_time))

                if r == int(len(prep) - 1):
                    print('100% completed')
                    print("--- %s seconds ---" % (T.time() - start_time))

                row = prep.iloc[r]
                features = compute_features(row)

                #
                feat[r, :] = features.reshape(1, -1)

            prep["ShoulderSlope_no_drct"] = ""
            prep["CurveArea"] = ""
            prep["TrunkAng"] = ""
            prep["AnkleDistance"] = ""

            prep["ShoulderSlope_up_down"] = ""
            prep["ShoulderSlope_right_up"] = ""
            prep["ShoulderSlope_front_back"] = ""
            prep["ShoulderSlope_right_front"] = ""

            prep["AnkleDistance_right_front"] = ""

            prep["Hip_wrist_dist_right"] = ""
            prep["Hip_wrist_dist_left"] = ""
            prep["COM_front_back"] = ""
            prep["COM_left_right"] = ""

            #
            prep.loc[:, "ShoulderSlope_no_drct":"COM_left_right"] = feat

            # stack direction and angles
            prep["ShoulderSlope_up_down"] = prep["ShoulderSlope_up_down"] * prep["ShoulderSlope_right_up"]
            prep["ShoulderSlope_front_back"] = prep["ShoulderSlope_front_back"] * prep["ShoulderSlope_right_front"]
            prep["AnkleDistance_front_back"] = prep["AnkleDistance"] * prep["AnkleDistance_right_front"]

            # final
            final = prep[["ShoulderSlope_up_down", "ShoulderSlope_front_back", "CurveArea", "TrunkAng",
                          # "AnkleDistance_front_back"]]

                          # final = prep[["frame_org", "ShoulderSlope_up_down", "ShoulderSlope_front_back", "CurveArea", "TrunkAng",
                          "AnkleDistance_front_back", "Hip_wrist_dist_right", "Hip_wrist_dist_left", "COM_front_back",
                          "COM_left_right"]]
            
            from scipy.signal import medfilt

            # assign NaN values to rows where the absolute value of ShoulderSlope_up_down is greater than 75
            final.loc[numpy.abs(final["ShoulderSlope_up_down"]) > 75, :] = numpy.nan
            final.loc[numpy.abs(final["CurveArea"]) > 100, :] = numpy.nan

            # apply the median filter to the specified columns
            cols_to_filter = ["ShoulderSlope_up_down", "ShoulderSlope_front_back", "CurveArea", "TrunkAng",
                              "AnkleDistance_front_back", "Hip_wrist_dist_right", "Hip_wrist_dist_left",
                              "COM_front_back", "COM_left_right"]
            final[cols_to_filter] = final[cols_to_filter].apply(lambda x: medfilt(x, kernel_size=5), axis=0)

            # specify path and file names

            # write
            final.to_csv(os.path.join(output_path, file_name))

In [41]:
ms3_FeatureCompute('/Users/fjonalutaj/Desktop/processed_data/', '/Users/fjonalutaj/Desktop/features/')

Processing file: 20240305_120255.csv
10% completed
--- 0.268887996673584 seconds ---
25% completed
--- 0.654428243637085 seconds ---
50% completed
--- 1.302011251449585 seconds ---
75% completed
--- 1.9519431591033936 seconds ---
100% completed
--- 2.5688319206237793 seconds ---
Processing file: 20240305_120655.csv


/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:565: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["ShoulderSlope_up_down"]) > 75, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:566: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["CurveArea"]) > 100, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:572: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

10% completed
--- 0.250532865524292 seconds ---
25% completed
--- 0.6356050968170166 seconds ---
50% completed
--- 1.2725629806518555 seconds ---
75% completed
--- 1.899953842163086 seconds ---
100% completed
--- 2.523798704147339 seconds ---
Processing file: 20240305_120424.csv
10% completed
--- 0.16693401336669922 seconds ---


/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:565: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["ShoulderSlope_up_down"]) > 75, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:566: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["CurveArea"]) > 100, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:572: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

25% completed
--- 0.42993688583374023 seconds ---
50% completed
--- 0.8561708927154541 seconds ---
75% completed
--- 1.285740852355957 seconds ---
100% completed
--- 1.707711935043335 seconds ---
Processing file: 20240305_121256.csv


/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:565: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["ShoulderSlope_up_down"]) > 75, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:566: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["CurveArea"]) > 100, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:572: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

10% completed
--- 0.35447216033935547 seconds ---
25% completed
--- 0.899299144744873 seconds ---
50% completed
--- 1.7964119911193848 seconds ---
75% completed
--- 2.6732842922210693 seconds ---
100% completed
--- 3.518357038497925 seconds ---


/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:565: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["ShoulderSlope_up_down"]) > 75, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:566: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[numpy.abs(final["CurveArea"]) > 100, :] = numpy.nan
/var/folders/8t/29wnp4q11fgb1mrfs2pm5gp00000gn/T/ipykernel_15515/1194104612.py:572: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead